In [1]:
import numpy as np

In [2]:
from load_contracts import read_contract, preprocess_text
from load_info import get_security_names

In [3]:
filename = "contracts/135_ActelisNetworks_COI_01072005.pdf"

In [4]:
text = read_contract(filename)

In [5]:
text = text.split(" ")

In [20]:
def find_target(text, target):
    indicies = []
    removed = 0
    while True:
        try:
            for i, word in list(enumerate(target)):
                curr_index = text.index(word)
                if i == 0: #marks first index
                    print("i == 0")
                    start_index = curr_index
                    if len(target) == 1:
                        print("length 1")
                        indicies += [start_index + removed]
                    removed += start_index
                    text = text[start_index + 1:]
                    break
                if curr_index != (start_index + i): #skips words between 
                    print("invalid sequence", start_index)
                    removed += start_index
                    text = text[start_index + 1:]
                    break
                if i == len(target) - 1: #final element
                    if curr_index == (len(target) + start_index - 1):
                        print("valid sequence", start_index + removed)
                        indicies += [start_index + removed]
                    removed += start_index
                    text = text[start_index + 1:]
        except ValueError:
            return indicies
        
            
        
#             start_index 
#             try:
#                 cur_index = text.index(word)
#             except ValueError:
#                 return index

In [48]:
def find_target(text, target):
    target_loc = 0
    indicies = []
    start_index = 0
    for i in range(len(text)):
        word = text[i]
#         print("target_loc", target_loc)
        if word == target[target_loc]:
            if target_loc == 0: #marks start of sequence
                start_index = i
            target_loc += 1
            if target_loc == len(target):
                indicies += [start_index]
                target_loc = 0
        elif word == target[0]:
            target_loc = 1
        else:
            if target_loc > 0:
                i -= (target_loc - 1)
            target_loc = 0
    return indicies

In [49]:
x = find_target(text, ["common"])

In [50]:
x

[254,
 273,
 458,
 479,
 483,
 971,
 1002,
 1019,
 1207,
 1251,
 1305,
 1893,
 1922,
 2082,
 2102,
 2363,
 2385,
 2704,
 2740,
 2864,
 2986,
 3114,
 3375,
 3457,
 3515,
 3727,
 3800,
 3844,
 3937,
 4054,
 4128,
 4154,
 4353,
 4363,
 4463,
 4473,
 4536,
 4875,
 4886,
 4915,
 4979,
 4994,
 5096,
 5116,
 5372,
 5521,
 5699,
 5779,
 5827,
 5846,
 5942,
 6125,
 6139,
 6185,
 6197,
 6287,
 6315,
 6339,
 6370,
 6382,
 6413,
 6421,
 6431,
 6516,
 6567,
 6588,
 6629,
 6784,
 6812,
 6852,
 6988,
 7146,
 7400,
 7619,
 7642,
 7692,
 7759,
 8117,
 8125,
 8147,
 8184,
 8204,
 8254,
 8376,
 8427,
 8440,
 8478,
 8789,
 8852,
 8886,
 9146,
 9177,
 9207,
 9233,
 9873,
 10209,
 10254,
 10336,
 10350,
 10550,
 10694,
 10707]

In [9]:
unsplit_names = get_security_names()

In [10]:
names = [name.split(" ") for name in unsplit_names]

In [11]:
indicies = []
for name in names:
    indicies += [find_target(text, name)]


In [12]:
#text

In [51]:
index_of_article_IV = find_target(text, ["articl", "fourth"])

In [58]:
text[index_of_article_IV[0]: index_of_article_IV[0] + 2]

['articl', 'fourth']

In [61]:
text[index_of_article_IV: 100]

TypeError: slice indices must be integers or None or have an __index__ method

In [ ]:
index_end_intro_IV = find_target(text, [""])